Selección de la base de datos

Se utilizará la base de datos https://www.kaggle.com/datasets/shaikhabdulrafay03/student-data, la cual contiene datos de una encuesta respondida por alumnos de distintas universidades, en la cual se pidió detallar elementos como: universidad a la que asiste, carrera, tiempo de estudio por día y educación de los padres, entre otras.


El objetivo de la red a realizar es, a partir de los datos provistos, poder predecir el rendimiento académico de los alumnos, y determinar qué características influyen en dichos resultados.

*Tipos de datos:
-Horario de encuesta
-Nombre de usuario
-Universidad
-Carrera
-Asistencia promedio a las clases
-Tiempo promedio de estudio por día
-Actividades extracurriculares
-Tiempo de descanso 
-Número de familiares
-Actividad física
-Actividad desarrollada en tiempo libre
-Miembro de una comunidad universitaria
-Estudios máximos alcanzados por los padres
-Nota media final

ANOTACIONES

-El tipo de carrera puede estar relacionado con la nota final (carreras mas dificiles pueden hacer que las notas sean bajas)
-Las variables pueden pasarse a valores entre 0 y 1 para trabajarlas más fácil